In [10]:
import numpy as np
import pandas as pd
import copy
import csv
import re 
import sys
import os
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset

In [2]:
#load winobias data
def load_winobias_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    sentences = [clean_winobias_text(line.strip()) for line in lines]
    return sentences

def clean_winobias_text(text):
    # Remove line numbers (e.g., "1. ", "2. ")
    text = re.sub(r'^\d+\s*', '', text)
    # Remove brackets (e.g., "[The nurse]" → "The nurse")
    text = re.sub(r'\[(.*?)\]', lambda x: x.group(1), text)
    return text.strip()

In [3]:
biased = load_winobias_data('winobias_data/pro_stereotyped_type1.txt.dev')
print(biased[:5])

anti_biased = load_winobias_data('winobias_data/anti_stereotyped_type1.txt.dev')
print(anti_biased[:5])


['The developer argued with the designer because he did not like the design.', 'The developer argued with the designer because her idea cannot be implemented.', 'The mechanic gave the clerk a present because he won the lottery.', 'The mechanic gave the clerk a present because it was her birthday.', 'The mover said thank you to the housekeeper because he is grateful.']
['The developer argued with the designer because she did not like the design.', 'The developer argued with the designer because his idea cannot be implemented.', 'The mechanic gave the clerk a present because she won the lottery.', 'The mechanic gave the clerk a present because it was his birthday.', 'The mover said thank you to the housekeeper because she is grateful.']


In [4]:
#create winobias dataframe

df_biased = pd.DataFrame({"text": biased, "label": 1})
df_anti_biased = pd.DataFrame({"text": anti_biased, "label": 0})

df = pd.concat([df_biased, df_anti_biased], ignore_index=True)

#shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

print(df.head())



                                                text  label
0  The salesperson chatted with the cleaner becau...      0
1  The cook is always teaching the assistant new ...      0
2  The mover moved the desk for the cleaner becau...      0
3  The salesperson sold some books to the librari...      0
4  The CEO gave the cashier cash because he wante...      1


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class WinobiasDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

In [6]:
dataset = WinobiasDataset(df["text"].tolist(),df["label"].tolist(),tokenizer,)

In [7]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

train_dataset = WinobiasDataset(train_df["text"].tolist(),train_df["label"].tolist(),tokenizer,)
test_dataset = WinobiasDataset(test_df["text"].tolist(),test_df["label"].tolist(),tokenizer,)

In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    eval_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.694754
2,No log,0.702511
3,No log,0.695537
4,No log,0.707431
5,No log,0.662932
6,No log,0.629631
7,0.636200,0.746404
8,0.636200,0.618073
9,0.636200,0.776543
10,0.636200,0.758271


TrainOutput(global_step=800, training_loss=0.45413774967193604, metrics={'train_runtime': 160.3612, 'train_samples_per_second': 39.473, 'train_steps_per_second': 4.989, 'total_flos': 416373245107200.0, 'train_loss': 0.45413774967193604, 'epoch': 10.0})

In [17]:
# Predict on a new sentence
def predict_bias(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    model.to(device)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).to("cpu")
    return "Biased" if torch.argmax(probs) == 1 else "Unbiased"

print(predict_bias("The nurse said she would help."))  # Likely "Biased"
print(predict_bias("The doctor said they would help."))  # Likely "Unbiased"

Unbiased
Unbiased


In [18]:
from transformers import pipeline
nlp = pipeline("text-classification", model="wu981526092/Sentence-Level-Stereotype-Detector", tokenizer="wu981526092/Sentence-Level-Stereotype-Detector")

print(nlp("The nurse said she would help."))  # Likely "Biased"
print(nlp("The doctor said they would help."))  # Likely "Unbiased"

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

c:\Users\Yuan Qi\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yuan Qi\.cache\huggingface\hub\models--wu981526092--Sentence-Level-Stereotype-Detector. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'stereotype_profession', 'score': 0.5301551818847656}]
[{'label': 'anti-stereotype_profession', 'score': 0.5276275873184204}]


In [ ]:
#test on winobias data

biased_test = load_winobias_data('winobias_data/pro_stereotyped_type1.txt.test')
anti_biased_test = load_winobias_data('winobias_data/anti_stereotyped_type1.txt.test')



df_biased_test = pd.DataFrame({"text": biased_test, "label": 1})
df_anti_biased_test = pd.DataFrame({"text": anti_biased_test, "label": 0})

df_test = pd.concat([df_biased_test, df_anti_biased_test], ignore_index=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

test_dataset = WinobiasDataset(df_test["text"].tolist(),df_test["label"].tolist(),tokenizer,)




{'eval_loss': 2.3147127628326416,
 'eval_runtime': 5.0367,
 'eval_samples_per_second': 157.246,
 'eval_steps_per_second': 19.656,
 'epoch': 10.0}

In [ ]:
import gensim
import numpy as np

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def same_score(word, attribute_set, model):
    """Compute the mean cosine similarity between a word and an attribute set."""
    similarities = [cosine_similarity(model[word], model[attr]) for attr in attribute_set if attr in model]
    return np.mean(similarities) if similarities else 0

def delta_same(target_set, attribute_set_a, attribute_set_b, model):
    """Compute the bias score between two attribute sets using SAME."""
    scores_a = [same_score(word, attribute_set_a, model) for word in target_set if word in model]
    scores_b = [same_score(word, attribute_set_b, model) for word in target_set if word in model]
    return np.mean(scores_a) - np.mean(scores_b) if scores_a and scores_b else 0

def compute_gender_direction(gender_pairs, model):
    """Compute the gender direction based on word pairs."""
    differences = [model[male] - model[female] for male, female in gender_pairs if male in model and female in model]
    return np.mean(differences, axis=0)

def direct_bias(word, gender_direction, model):
    """Compute the direct bias of a word with respect to gender direction."""
    if word not in model:
        return None
    return abs(cosine_similarity(model[word], gender_direction))

# Example Usage
# Load a pre-trained Word2Vec model (change the path as needed)
model = gensim.models.KeyedVectors.load_word2vec_format("word2vec/GoogleNews-vectors-negative300.bin.gz", binary=True)

# Define word sets
target_words = ["engineer", "scientist", "doctor", "nurse"]
male_attributes = ["he", "him", "his", "John"]
female_attributes = ["she", "her", "hers", "Mary"]

gender_pairs = [("he", "she"), ("man", "woman"), ("king", "queen"), ("brother", "sister")]

# Compute gender direction
gender_direction = compute_gender_direction(gender_pairs, model)

# Compute direct bias for target words
for word in target_words:
    bias = direct_bias(word, gender_direction, model)
    print(f"Direct Bias of {word}: {bias}")

# Compute bias score
bias_score = delta_same(target_words, male_attributes, female_attributes, model)
print(f"Gender Bias Score (ΔSAME): {bias_score}")


Direct Bias of engineer: 0.10186593234539032
Direct Bias of scientist: 0.014988328330218792
Direct Bias of doctor: 0.03561718761920929
Direct Bias of nurse: 0.338949590921402
Gender Bias Score (ΔSAME): -0.030900567770004272
